In [1]:
import sys
import os
import intake
import xarray as xr
import numpy as np
import pandas as pd
import cftime

from dask.distributed import Client

client = Client()

In [2]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)

In [3]:
import pprint

uni_dict = col.unique(["experiment_id"]) # source_id, "", "table_id", "variable_id"])
#pprint.pprint(uni_dict, compact=True)

In [4]:
### Choose simulation ###

exp = "ssp245"  # "historical"  # "ssp126"  #  "piControl"  # "1pctCO2"  #  "abrupt-4xCO2"  # "rcp85-cmip5" # "amip"  # "hist-nat" # "ssp585"  # "ssp585"  # "historical-cmip5" # "historical" # "hist-CO2"  #   # "hist-GHG"  # "hist-nat"  # 'amip-hist'
source = "CNRM-ESM2-1"  # "CNRM-CM6-1"  # "CanESM5"  # "IPSL-CM6A-LR"  # "ACCESS-ESM1-5"  # "MPI-ESM1-2-LR"  # "EC-Earth3"  # "UKESM1-0-LL"  # "SAM0-UNICON"  # "MIROC-ES2L"  # "BCC-ESM1"  # "GFDL-ESM4"  # "FGOALS-f3-L"
# "CMCC-ESM2"  # "FGOALS-g3"  # "HadGEM3-GC31-LL"  # "NorESM2-LM" # "MRI-ESM2-0"  #  "BCC-CSM2-MR"  # "CESM2" # "MIROC6"  # "INM-CM5-0"  # "GFDL-CM4"  # "IPSL-CM5A-LR"  # CESM1-1-CAM5-CMIP5 # "IPSL-CM6A-ATM-HR"  # "HadGEM3-GC31-MM"
var = "pr"           # variable
table_id = "day"  # "Amon" #"day", #"3hr",  #Eday
exp, source, var, table_id

('ssp245', 'CNRM-ESM2-1', 'pr', 'day')

In [5]:
cat = col.search(
    experiment_id=[exp],  # ssp126
    table_id=table_id,
    variable_id=var,
    #member_id='r1i1p1f1', #['r7i1p1f1', 'r8i1p1f1'], # 'r1i1p1f1', 'r2i1p1f1', 'r4i1p1f1', 'r6i1p1f1'],  #, , 'r9i1p1f1', 'r10i1p1f1', 'r11i1p1f1', 'r12i1p1f1', 'r13i1p1f1'],
    source_id=[source]
)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,CNRM-CERFACS,CNRM-ESM2-1,ssp245,r1i1p1f2,day,pr,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,NaN,20190328


In [6]:
data_source = cat[list(cat.keys())[0]]
ds = data_source(
    zarr_kwargs={"consolidated": True, "decode_times": True}
).to_dask()

In [7]:
members = ds.member_id
members

# members treatment for CanESM5
if source == "CanESM5":
    members_ = np.array(members)
    mems = [mem for mem in members_]
    mems_ = [mem for mem in mems if 'i1p1f1' in mem]
    #mems_ = mems_[0:10]
elif source == "MIROC6":
    mems_ = ['r8i1p1f1', 'r9i1p1f1', 'r4i1p1f1', 'r5i1p1f1', 'r7i1p1f1', 'r1i1p1f1', 'r2i1p1f1', 'r6i1p1f1', 'r3i1p1f1', 'r10i1p1f1', 'r11i1p1f1',
    'r14i1p1f1', 'r15i1p1f1', 'r12i1p1f1', 'r18i1p1f1', 'r25i1p1f1', 'r26i1p1f1', 'r27i1p1f1', 'r20i1p1f1', 'r21i1p1f1', 'r13i1p1f1', 'r16i1p1f1', 
    'r17i1p1f1', 'r19i1p1f1', 'r22i1p1f1', 'r24i1p1f1', 'r23i1p1f1', 'r30i1p1f1', 'r29i1p1f1', 'r28i1p1f1']
    mems_ = xr.DataArray(mems_)
elif (source == 'ACCESS-ESM1-5') and (exp == 'ssp245'):   # TO COMPLETE DOWNLOAD
    mems_ = members[3:]
else:
    mems_ = members

if table_id == "Amon":
    mems_ = [mems_[0]]
else:
    mems_ = mems_

mems_

<xarray.DataArray 'member_id' (member_id: 6)>
array(['r3i1p1f1', 'r2i1p1f1', 'r1i1p1f1', 'r14i1p1f1', 'r6i1p1f1', 'r4i1p1f1'],
      dtype='<U9')
Coordinates:
  * member_id  (member_id) <U9 'r3i1p1f1' 'r2i1p1f1' ... 'r6i1p1f1' 'r4i1p1f1'

In [7]:
mems_ = xr.DataArray(['r11i1p1f1', 'r12i1p1f1', 'r13i1p1f1', 'r14i1p1f1', 'r15i1p1f1', 'r16i1p1f1', 'r17i1p1f1', 'r18i1p1f1', 'r19i1p1f1', 'r20i1p1f1', 'r21i1p1f1', 'r22i1p1f1', 'r23i1p1f1', 'r24i1p1f1', 'r25i1p1f1'])

In [8]:
pr = ds.pr   # SET VARIABLE
lats = np.array(pr.lat)
lons = np.array(pr.lon)
lat_diffs = [lats[ilat+1]-lats[ilat] for ilat, lat in enumerate(lats[:-1])]
lon_diffs = [lons[ilon+1]-lons[ilon] for ilon, lon in enumerate(lons[:-1])]
lat_diff_mean = np.mean(lat_diffs)
lat_res = np.round(lat_diff_mean, 2)
lon_diff_mean = np.mean(lon_diffs)
lon_res = np.round(lon_diff_mean, 2)
print('Hor. Res.: {0}x{1}'.format(str(lat_res), str(lon_res)))

Hor. Res.: 2.79x2.81


In [9]:
pr.indexes['time']

CFTimeIndex([1850-01-01 12:00:00, 1850-01-02 12:00:00, 1850-01-03 12:00:00,
             1850-01-04 12:00:00, 1850-01-05 12:00:00, 1850-01-06 12:00:00,
             1850-01-07 12:00:00, 1850-01-08 12:00:00, 1850-01-09 12:00:00,
             1850-01-10 12:00:00,
             ...
             2014-12-22 12:00:00, 2014-12-23 12:00:00, 2014-12-24 12:00:00,
             2014-12-25 12:00:00, 2014-12-26 12:00:00, 2014-12-27 12:00:00,
             2014-12-28 12:00:00, 2014-12-29 12:00:00, 2014-12-30 12:00:00,
             2014-12-31 12:00:00],
            dtype='object', name='time', length=60225)

In [10]:
if (exp == "piControl") or ('ssp' in exp) or (exp == '1pctCO2') or (exp == 'abrupt-4xCO2') or (exp == 'amip-4xCO2'):
    times = pr.indexes['time']
    years = times.year
    years = np.unique(years)
    years_ = [str(year) for year in years]
elif exp == 'historical':   
    y0 = 1950  # 1850  DO NOT DOWNLOAD FULL HISTORICAL PERIOD !!!!!!!!!!!
    y1 = 2014
    years = np.arange(y0, y1+1, 1)
    years_ = [str(y) for y in years]
else:
    print("Don't know what to do with {0}".format(exp))
    sys.exit()
    
"""elif 'ssp' in exp:
    times = pr.indexes['time']
    years = times.year
    years = np.unique(years)
    years_ = [str(year) for year in years]"""

    
wd = os.getcwd()

datadir = wd + '/data'

if not os.path.isdir(datadir + '/' + source):
    os.mkdir(datadir + '/' + source)
outdir = datadir + '/' + source

if not os.path.isdir(outdir + '/' + exp):
    os.mkdir(outdir + '/' + exp)
outdir = outdir + '/' + exp

res_ = str(lat_res) + "x" + str(lon_res)

print(outdir)


for mem in mems_:
    mem_ = str(mem.values)

    print('\nMember id: {0}'.format(mem_))
    
    if not os.path.isdir(outdir + '/' + mem_):
        os.mkdir(outdir + '/' + mem_)
    outdir_ = outdir + '/' + mem_
    
    if not os.path.isdir(outdir_ + '/' + res_):
        os.mkdir(outdir_ + '/' + res_)
    outdir_ = outdir_ + '/' + res_
    
    if not os.path.isdir(outdir_ + '/pr'):
        os.mkdir(outdir_ + '/pr')
    outdir_ = outdir_ + '/pr'
    
    k = 0

    for y in years_:
        print(y, end=' : ', flush=True)
        
        ly = len(y)
        
        if ly < 4:
            y = (4 - ly) * '0' + y
            
        pr = ds.pr.sel(member_id=mem).sel(time=slice(y,y)).load()  # ne marche pas avec CESM2: '1' -> '0001'

        if (source == 'CESM2' or source == 'ACCESS-ESM1-5' or source == 'GFDL-CM4') and (exp == 'abrupt-4xCO2' or exp == 'piControl' or exp == '1pctCO2'):
            #l = len(pr.indexes['time'])
            #y_ = str(int(y)+1199)
            dates = [cftime.datetime(d.year, d.month, d.day) for d in pr.indexes['time']]  # .isoformat()

            assert len(dates) == len(pr.indexes['time']), 'wrong dates length'
            
            dates = xr.CFTimeIndex(dates)
            #dates = np.array('{0}-01-01'.format(y), dtype=np.datetime64) + np.arange(l)
            #dates = xr.cftime_range(start='{0}-01-01'.format(y), end='{0}-12-31'.format(y), calendar='julian')

            #dates = pd.DatetimeIndex(dates)
            pr['time'] = dates
        
        elif (source == 'CESM2' or source == 'SAM0-UNICON') and (exp != 'piControl' and exp != 'abrupt-4xCO2'):
            pr['time'] = pr.indexes['time'].to_datetimeindex()

        # change coordinates (0-360) -> (-180,180)
        pr_rolled = pr.roll(lon=int(pr.lon.size/2), roll_coords=True)
        pr_rolled = pr_rolled.assign_coords(lon=np.linspace(pr_rolled.lon[0]-360, pr_rolled.lon[-1], pr.lon.size))   # coordinates 0-360° -> -180-180°

        prday = pr_rolled * 86400 # 3hr mean precipitation flux: kg/m²/s -> mm/day (1day = 86400s)

        outfile = outdir_ + '/' + y + '.nc'
        
        prday.to_netcdf(outfile)
        
        k = k+1

print('\nDone')

/home/guillaumechagnaud/CMIP6/scripts/data/CanESM5/historical

Member id: r11i1p1f1
1950 : 1951 : 1952 : 1953 : 1954 : 1955 : 1956 : 1957 : 1958 : 1959 : 1960 : 1961 : 1962 : 1963 : 1964 : 1965 : 1966 : 1967 : 1968 : 1969 : 1970 : 1971 : 1972 : 1973 : 1974 : 1975 : 1976 : 1977 : 1978 : 1979 : 1980 : 1981 : 1982 : 1983 : 1984 : 1985 : 1986 : 1987 : 1988 : 1989 : 1990 : 1991 : 1992 : 1993 : 1994 : 1995 : 1996 : 1997 : 1998 : 1999 : 2000 : 2001 : 2002 : 2003 : 2004 : 2005 : 2006 : 2007 : 2008 : 2009 : 2010 : 2011 : 2012 : 2013 : 2014 : 
Member id: r12i1p1f1
1950 : 1951 : 1952 : 1953 : 1954 : 1955 : 1956 : 1957 : 1958 : 1959 : 1960 : 1961 : 1962 : 1963 : 1964 : 1965 : 1966 : 1967 : 1968 : 1969 : 1970 : 1971 : 1972 : 1973 : 1974 : 1975 : 1976 : 1977 : 1978 : 1979 : 1980 : 1981 : 1982 : 1983 : 1984 : 1985 : 1986 : 1987 : 1988 : 1989 : 1990 : 1991 : 1992 : 1993 : 1994 : 1995 : 1996 : 1997 : 1998 : 1999 : 2000 : 2001 : 2002 : 2003 : 2004 : 2005 : 2006 : 2007 : 2008 : 2009 : 2010 : 2011 : 2012 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [22]:
outfile

'/home/chagnaug/CMIP6/data/HadGEM3-GC31-LL/historical/r4i1p1f3/1.25x1.88/pr/2014.nc'

In [18]:
pr['time']

<xarray.DataArray 'time' (time: 366)>
array([cftime.datetime(151, 1, 1, 0, 0, 0, 0),
       cftime.datetime(151, 1, 1, 0, 0, 0, 0),
       cftime.datetime(151, 1, 2, 0, 0, 0, 0), ...,
       cftime.datetime(151, 12, 29, 0, 0, 0, 0),
       cftime.datetime(151, 12, 30, 0, 0, 0, 0),
       cftime.datetime(151, 12, 31, 0, 0, 0, 0)], dtype=object)
Coordinates:
  * time       (time) object 0151-01-01 00:00:00 ... 0151-12-31 00:00:00
    member_id  <U8 'r1i1p1f1'

In [14]:
test = xr.cftime_range(start='1201-01-01', end='1201-12-31', calendar='all_leap')
len(test)

366

In [18]:
#dates = np.array('{0}-01-01'.format(y), dtype=np.datetime64) + np.arange(365)
dates = [cftime.datetime(d.year,d.month,d.day) for d in pr.indexes['time']] # .isoformat()
test2 = xr.CFTimeIndex(dates)

In [21]:
pr.indexes['time']

CFTimeIndex([0004-01-01 00:00:00, 0004-01-02 00:00:00, 0004-01-03 00:00:00,
             0004-01-04 00:00:00, 0004-01-05 00:00:00, 0004-01-06 00:00:00,
             0004-01-07 00:00:00, 0004-01-08 00:00:00, 0004-01-09 00:00:00,
             0004-01-10 00:00:00,
             ...
             0004-12-22 00:00:00, 0004-12-23 00:00:00, 0004-12-24 00:00:00,
             0004-12-25 00:00:00, 0004-12-26 00:00:00, 0004-12-27 00:00:00,
             0004-12-28 00:00:00, 0004-12-29 00:00:00, 0004-12-30 00:00:00,
             0004-12-31 00:00:00],
            dtype='object', name='time', length=365)

In [15]:
type(prwa.indexes['time'])

xarray.coding.cftimeindex.CFTimeIndex

In [12]:
test = xr.cftime_range(start='3049-01-01', end='3049-12-31', calendar='noleap')
#test[0]
#test_ = test.to_datetimeindex()

In [26]:
pr_ = ds.pr.sel(member_id=mem).sel(time=slice(y,y)).load()

In [27]:
pr_['time'] = test

In [28]:
pr.indexes['time']

DatetimeIndex(['1754-08-30 22:43:41.128654848',
               '1754-08-31 22:43:41.128654848',
               '1754-09-01 22:43:41.128654848',
               '1754-09-02 22:43:41.128654848',
               '1754-09-03 22:43:41.128654848',
               '1754-09-04 22:43:41.128654848',
               '1754-09-05 22:43:41.128654848',
               '1754-09-06 22:43:41.128654848',
               '1754-09-07 22:43:41.128654848',
               '1754-09-08 22:43:41.128654848',
               ...
               '1755-08-20 22:43:41.128654848',
               '1755-08-21 22:43:41.128654848',
               '1755-08-22 22:43:41.128654848',
               '1755-08-23 22:43:41.128654848',
               '1755-08-24 22:43:41.128654848',
               '1755-08-25 22:43:41.128654848',
               '1755-08-26 22:43:41.128654848',
               '1755-08-27 22:43:41.128654848',
               '1755-08-28 22:43:41.128654848',
               '1755-08-29 22:43:41.128654848'],
              dtype=

mem